In [1]:
import os
import tensorflow as tf
#import pandas
from tensorflow.keras.applications import InceptionV3

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout,Input,Flatten,Dense,MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # Data Augumentation

In [ ]:
tf.test.is_gpu_available()

In [2]:
#batchsize=8
BATCH_SIZE = 4
EPOCHS = 2

In [3]:
train_datagen= ImageDataGenerator(rescale=1./255, rotation_range=0.2,shear_range=0.2,
    zoom_range=0.2,width_shift_range=0.2,
    height_shift_range=0.2, validation_split=0.2)

train_data= train_datagen.flow_from_directory(r'D:\Desktop\New folder\Driver-Drowsiness-Detection-using-Deep-Learning\Image\prepare_data\train',
                                target_size=(80,80),batch_size=BATCH_SIZE,class_mode='categorical',subset='training' )

validation_data= train_datagen.flow_from_directory(r'D:\Desktop\New folder\Driver-Drowsiness-Detection-using-Deep-Learning\Image\prepare_data\train',
                                target_size=(80,80),batch_size=BATCH_SIZE,class_mode='categorical', subset='validation')

Found 33557 images belonging to 2 classes.
Found 8389 images belonging to 2 classes.


In [4]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_data = test_datagen.flow_from_directory(r'D:\Desktop\New folder\Driver-Drowsiness-Detection-using-Deep-Learning\Image\prepare_data\train',
                                target_size=(80,80),batch_size=BATCH_SIZE,class_mode='categorical')

Found 41946 images belonging to 2 classes.


In [5]:
bmodel = InceptionV3(include_top=False, weights='imagenet', input_tensor=Input(shape=(80,80,3)))
hmodel = bmodel.output
hmodel = Flatten()(hmodel)
hmodel = Dense(64, activation='relu')(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(2,activation= 'softmax')(hmodel)

model = Model(inputs=bmodel.input, outputs= hmodel)
for layer in bmodel.layers:
    layer.trainable = False

In [ ]:
#model.summary()

In [6]:
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping, ReduceLROnPlateau


checkpoint = ModelCheckpoint(os.path.join(r'D:\Desktop\New folder\Driver-Drowsiness-Detection-using-Deep-Learning\Image', "model.h5"),
                            monitor = 'val_loss', save_best_only = True, verbose = 3)

earlystop = EarlyStopping(monitor = 'val_loss', patience = 7, 
                          verbose= 3, restore_best_weights = True)


In [7]:
learning_rate = ReduceLROnPlateau(monitor= 'val_loss', patience=3, verbose= 3, )

callbacks = [checkpoint, earlystop, learning_rate]

In [8]:
model.compile(optimizer = 'Adam', 
              loss = 'categorical_crossentropy', 
              metrics = ['accuracy'])

In [13]:
model.fit_generator(train_data,steps_per_epoch = train_data.samples// BATCH_SIZE,
                   validation_data = validation_data,
                   validation_steps = validation_data.samples// BATCH_SIZE,
                   callbacks = callbacks,
                    epochs = EPOCHS)

C:\Users\Ankul\AppData\Local\Temp/ipykernel_32316/3082670940.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_data,steps_per_epoch = train_data.samples// BATCH_SIZE,


AttributeError: module 'pandas' has no attribute 'Series'

In [ ]:
acc_tr, loss_tr = model.evaluate_generator(train_data)
print(acc_tr)
print(loss_tr)

In [ ]:
acc_vr, loss_vr = model.evaluate_generator(validation_data)
print(acc_vr)
print(loss_vr)

In [ ]:
acc_test, loss_test = model.evaluate_generator(test_data)
print(acc_tr)
print(loss_tr)